In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from ray.rllib.agents.trainer import Trainer, with_common_config
from ray.rllib.utils.annotations import override


In [ ]:
import ray
from ray import tune
from ray.rllib.utils.seed import seed as rllib_seed
import rl_toy
from rl_toy.envs import RLToyEnv
from ray.tune.registry import register_env
register_env("RLToy-v0", lambda config: RLToyEnv(config))

# rllib_seed(0, 0, 0)
ray.init()


In [ ]:
state_space_sizes = [8, 10, 12, 14] # [2**i for i in range(1,6)]
action_space_sizes = [2, 4, 8, 16] # [2**i for i in range(1,6)]
delays = [0] + [2**i for i in range(5)]
sequence_lengths = [i for i in range(1,6)]
reward_densities = [0.25] # np.linspace(0.0, 1.0, num=5)
# make_reward_dense = [True, False]
terminal_state_densities = [0.25] # np.linspace(0.1, 1.0, num=5)
algorithms = ["DQN"]

print('# Algorithm, state_space_size, action_space_size, delay, sequence_length, reward_density,'
               'terminal_state_density ')
print(algorithms, state_space_sizes, action_space_sizes, delays, sequence_lengths, reward_densities,
      terminal_state_densities)


In [ ]:
# average over 5-10 runs
# Keep minimum state size of 5 - so 1 terminal, 4 possible sequences of length 1 and 1 of them is rewarded - actually depends on densities
# min action size of 2; max action size up to twice of state space size; 
# delay looks ok
# sequence_lengths up to 6?

# Hyperparam search over RL algo. hyperparams, optimizer hyperparams; only for max sequence length and min delay?
# Hyperparams:



In [ ]:
# stats
# fout = open('rl_stats_temp.csv', 'a') #hardcoded
# fout.write('# basename, n_points, n_features, n_trees ')

# fout.close()

In [ ]:
import numpy as np
dir_name = '/home/rajanr/Backup/rl_toy'
dir_name = '/home/rajanr/custom-gym-env' #run
# stats_file = dir_name + '/' + 'lap_22' #Name of file to which benchmark stats were written
stats_file = dir_name + '/' + '128' #Name of file to which benchmark stats were written20
# stats_file = dir_name + '/' + 'rl_stats_temp_seeded_8.csv' #Name of file to which benchmark stats were written
# rl_stats_temp_initial.csv has initial working run: All dynamics functions were randomly generated with diff. seeds
# rl_stats_temp_seeded.csv has runs with fixed seeds for S and A, but not for Env. or Ray!!
# rl_stats_temp_seeded_2.csv has another run like ...seeded.csv
# rl_stats_temp_seeded_3.csv has same but for config space of S,A = 8,4 - so only 30 configs!
# rl_stats_temp_seeded_4.csv has same seed for Env. also now - further restricted space (delay up to 8, seq_len of 1 or 2) - only 10 configs!
# rl_stats_temp_seeded_5.csv same as 4 but seed for Ray would have differed
# rl_stats_temp_seeded_6.csv same as 4 and tried using rllib_seed for Ray
# rl_stats_temp_seeded_7.csv same as 4 and tried using rllib_seed for Ray
# rl_stats_temp_seeded_8.csv uses completely_connected config - so S,A = 8,8, non-repeating sequences, ray agent seed to set a seed for it.
# rl_stats_temp_seeded_9.csv same as 8
# 10.csv same as 8
# lap_1 removes "hiddens" layers
# lap_2 and lap_3 are same
# lap_4, lap_5, lap_6 have OHE
# lap_7, 8, 9, 10 have runs with diff. NN sizes - 12 configs with fixed del, seq_len of 0, 1
# lap_11, 12, 13 have fcnet_activation search - 3 configs and rest old "default" settings
# lap_14, 15, 16 have learning_starts and target_net_upd_freqs - 5 x 3 = 15 configs;
# lap_17, 18, 19 have double dqn tests - 2 configs
# lap_20, 21, 22 (3x3 configs) same as 16 but changed target_net_upd_freq grid based on the results that higher value of 800 seemed much better for 14,15,16, also removed 1st and last values of learning_starts as it didn't seem to have much of an effect
# 12.csv is with new target_nn_upd_freq but ran it for the 5x2 grid (of del x seq_len?) by mistake instead of NN size search
# 13, 14 (4x3 configs of layers x width) is with nn search and new learning_starts of 1000 and target_net_upd_freq of 800
# 15 changed widths to [128, 256, 512], rest same as 14
# --16, 17 (5x2 configs) same as 12 but with Ray config["seed"] set
# --18, 19 same as 16 (with config seed of Ray) but with ray.init(local_mode=True)
# 20, 21, 22 have 5x4 configs for learning_rates x adam_epsilon; 
# 23 same as 22 but changed adam_epsilon grid
# 24 MAIN EXPERIMENT: has (5x4x10 configs of del x seq_len x seed): main experiment with DQN
# 25 have (5x2x10 configs of del x seq_len x seed): main experiment with DQN; ran with wrong (smaller) grid for seq_len by mistake
# 26: MAIN EXPERIMENT: (5x4x10 configs of del x seq_len x seed): main experiment with DQN; same as 24
# 27: (5x4x10 configs of del x seq_len x seed): main experiment with DQN; same as 26 but with target_nn_upd_f=80 and learning_starts=2000 to see how poor hyperparam config. does
# +28: (1x10 configs of seeds for del, seq of 0, 1): rest same as 26 but for 100,000 steps
# +29: (1x10 configs of seeds for del, seq of 0, 1): rest same as 28 but for schedule_max_timesteps of 50,000
# 30: A3C (5x2x3 configs of del x seq x seed): has some defaults for A3C run over diff. del x seq configs.
# 31: ran 30 again by mistake
# 32: (4x3x3 configs of num_layer x layer_width x seed): for HPO of NN size, rest same as 30
# 33: (5x3x3 configs of lr x nn activation x seed): HPO, rest same as 32 (ran with NN of [256, 256] I think)
# 34: (5x3x3 configs of lr x nn activation x seed): HPO, changed NN to [128, 128, 128] based on # 32, rest same as 33
# 35: (4x3x3 configs of lambda x grad_clip x ...): HPO, keep lr and act. same based on 33 and 34: relu and sigmoid pretty bad in both
# 36: (3x4x3 configs of vf_loss_coeff x entropy_coeff): HPO, changed lambda and grad_clip based on 35 to 0, 10
# 37: MAIN EXPERIMENT: (5x4x10 of del x seq x seed): changed entropy_coeff to 0.1 based on 36
# 38: LSTM (5x2x3 of del x seq x seed): used max_seq_len of LSTM = del + seq_len, rest same as 37
# 39: (5x2x3 of del x seq x seed): changed lstm_use_prev_action_reward to True, rest same as 38
# 40: (4x3x3 configs of num_layer x layer_width x seed): for HPO of NN size, rest same as 39 (because it seemed to do noisily better than 38)
# 41: (5x3x3 configs of lr x nn activation x seed): HPO of lr and act., changed NN to [256, 256, 256, 256] based on 40
# 42: (4x3x3 configs of lambda x grad_clip x ...): HPO, keep lr and act. same based on 41: sigmoid pretty bad in both
# 43: (3x4x3 configs of vf_loss_coeff x entropy_coeff): HPO, changed lambda and grad_clip based on 42 to 1, 10
# 44: (3x2x3 configs of lstm cell size x ... params): HPO, changed vf_loss_coeff and entropy_coeff to 0.1 based on 43
# 45: MAIN EXPERIMENT: (5x4x10 of del x seq x seed): changed vf_loss_coeff and entropy_coeff to 0.1 based on 43
# 46-50: Rainbow: (1x1x3 configs of del x seq x seed): has some defaults for rainbow; Faced problems with num_steps_trained to num_steps_sampled ratio
# 51: (4x3x3 configs of num_layer x layer_width x seed): for HPO of NN size, train Batch size of 64, rest same as DQN best + some defaults for Rainbow.
# 52: Tune rainbow params 1st and THEN the others: (4x3x3 configs of alpha x beta x seed): for HPO of rainbow params set 2 (forogt set 1), train Batch size of 64, rest same as DQN best + some defaults for Rainbow.
# 53: (4x3x3 configs of n_step x num_atoms x seed): for HPO of rainbow params set 1, rest same as 52
# 54: (4x3x3 configs of alpha x beta x seed): for HPO of rainbow params set 2, set n_step to 4, rest same as 53; seems num_steps trained is 16 times more now.
# 55: (4x3x3 configs of num_layer x layer_width x seed): for HPO of NN size, train Batch size of 32, alpha = 0.75, rest same as 54
# 56: (1x3x3 configs of nn activation x seed): HPO of nn_act., changed NN to [256, 256] based on 56 (512 was only slightly better - 256 should suffice, num_layers did not have a big impact)
# 57: (1x5x3 configs of lr x seed): HPO of lr
# 58: (4x3x3 configs of learning_starts x target_net_upd_freq): HPO; learning rate changed to 1e-3 based on 57
# 59: MAIN EXPERIMENT: (5x4x10 of del x seq x seed): set learning_starts, target_net_upd_freq to 500, 80 based on 58
# 60-69: Noise: initial experiments with transition and reward noise. In 66, changed evaluation_config to contain 0 noise.
# 70: MAIN EXPERIMENT: Noises (6x6x10 of transition noise x reward noise)
# 71-: Sparsity: 
# 72-91: Runs to fix evaluation_config to get proper eval metrics with a hack
# 92: --MAIN EXPERIMENT: Sparsity: (1x4x10 of seq_len x seed): On best DQN config; eval results not proper due to Ray
# 93: --MAIN EXPERIMENT: Noises A3C: Noises (5x5x10 of transition noise x reward noise) INTerrupted
# 94: --MAIN EXPERIMENT: Noises A3C + LSTM: Noises (5x5x10 of transition noise x reward noise) INTerrupted
# 95: Had to hack in more stuff for proper eval metrics.
# 96: --MAIN EXPERIMENT: Sparsity: (1x3x10 of seq_len x seed): On best DQN config; eval horizon was unlimited!!
# 97: --MAIN EXPERIMENT: Noises A3C: Noises (5x5x10 of transition noise x reward noise) INTerrupted, taking too long!
# 98: Tried printing out eval rewards to make sure there are not multiple agents' rewards for A3C.
# 99: --MAIN EXPERIMENT: Noises A3C + LSTM: Noises (5x5x10 of transition noise x reward noise) INTerrupted, taking too long!
# 100: Tested horizon for eval metrics.
# 101: relaunched 97 with eval horizon 100; INTerrupted due to Python Bus error
# 102: relaunched 99 with eval horizon 100
# 103: MAIN EXPERIMENT: (5x5x10 of del x seq x seed): relaunched 70 with eval metrics, eval horizon 100 and (5x5x10 of transition noise x reward noise)
# 106: MAIN EXPERIMENT: (5x4x10 of del x seq x seed): relaunched 26 with eval metrics, eval horizon 100
# 110: --MAIN EXPERIMENT: Noises Rainbow: (5x5x10 of transition noise x reward noise), eval horizon 100; INTerrupted by OS due to memory consumption
# 111: MAIN EXPERIMENT: A3C (5x4x10 of del x seq x seed): relaunched 37 with eval metrics, eval horizon 100
# 112: MAIN EXPERIMENT: A3C + LSTM (5x4x10 of del x seq x seed): relaunched 45 with eval metrics, eval horizon 100
# 113: --MAIN EXPERIMENT: Noises Rainbow: (5x5x10 of transition noise x reward noise), eval horizon 100; relaunched 110 in parts due to memory leak issue
# 114: --MAIN EXPERIMENT: Rainbow (5x4x10 of del x seq x seed); relaunched 59 with eval metrics, eval horizon 100; in parts due to memory leak issue
# 115: MAIN EXPERIMENT: Noises A3C: (5x5x10 of transition noise x reward noise), eval horizon 100; relaunch of 97
# 116: MAIN EXPERIMENT: Noises A3C + LSTM: (5x5x10 of transition noise x reward noise), eval horizon 100; relaunch of 99
# 117: MAIN EXPERIMENT: Sparsity: (1x3x10 of seq_len x seed): On best DQN config; relaunch of 92
# 118: MAIN EXPERIMENT: Sparsity A3C: (1x3x10 of seq_len x seed): On best A3C config;
# 119: MAIN EXPERIMENT: Sparsity A3C + LSTM: (1x3x10 of seq_len x seed): On best A3C + LSTM config;
# 120: MAIN EXPERIMENT: Sparsity Rainbow: (1x3x10 of seq_len x seed): On best Rainbow config;
# 121: --MAIN EXPERIMENT: Rainbow (5x4x10 of del x seq x seed); Relaunched 114; apparently only ran 1st past of script, thanks to bash reloading from disk when running a new line
# 122: MAIN EXPERIMENT: Noises Rainbow (5x5x10 of transition noise x reward noise); Relaunched 113
# 123: MAIN EXPERIMENT: Rainbow (5x4x10 of del x seq x seed); Relaunched 114;
# python custom_agents_sparsity_2.py 124 | tee 124.log
# python custom_agents_a3c_sparsity_2.py 125 | tee 125.log
# python custom_agents_a3c_lstm_sparsity_2.py 126 | tee 126.log
# python custom_agents_rainbow_sparsity_2.py 127 | tee 127.log
# 128: MAIN EXPERIMENT: Increase A3C + LSTM sequence length to 10 times old one and check results. (Had a mistake: 10 * delay + seq_len: should have had bracket around whole expression!!)
# 129: MAIN EXPERIMENT: Launched # 129 with new Env seed for custom_agents.py to check effect of using a different env

# ICML re-runs:
# Some more in the 150s: need to check github commits for that
# 159: --MAIN EXPERIMENT: Relaunched 111: A3C (5x4x10 of del x seq x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 160: --MAIN EXPERIMENT: Relaunched 112: A3C + LSTM (5x4x10 of del x seq x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000; Mistake: ran it with "max_seq_len": 10 * delay + sequence_length (it was another mistake to have not put brackets around delay + seq.len. when multiplying by 10 even in the old exp.!!)
# 161: --MAIN EXPERIMENT: Relaunched 115: Noises A3C: (5x5x10 of transition noise x reward noise): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 162: --MAIN EXPERIMENT: Relaunched 116: Noises A3C + LSTM: (5x5x10 of transition noise x reward noise): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 163: --MAIN EXPERIMENT: Relaunched 160: A3C + LSTM (5x4x10 of del x seq x seed): with "max_seq_len": delay + sequence_length
# 164: Relaunched 163: with "max_seq_len": 10 * (delay + sequence_length)
# 165: --MAIN EXPERIMENT: Relaunched 118: Sparsity A3C: (1x3x10 of seq_len x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 166: --MAIN EXPERIMENT: Relaunched 119: Sparsity A3C + LSTM: (1x3x10 of seq_len x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 167: --MAIN EXPERIMENT: Relaunched 125: Sparsity 2 (changing reward densities) A3C (1x3x10 of rew dens x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000
# 168: --MAIN EXPERIMENT: Relaunched 126: Sparsity 2 (changing reward densities) A3C + LSTM (1x3x10 of rew dens x seed): with timesteps_per_iteration = 1000 and timesteps_total = 20000

# Final for paper: 103, 106, 111, 112, 115, 116, 117, 118, 119, 120, 123, more?
# Others are in /home/rajanr/Backup/rl_toy files

state_space_sizes = [8]#, 10, 12, 14] # [2**i for i in range(1,6)]
action_space_sizes = [8]#2, 4, 8, 16] # [2**i for i in range(1,6)]
delays = [0] + [2**i for i in range(4)]
sequence_lengths = [1, 2]#i for i in range(1,4)]
reward_densities = [0.25] # np.linspace(0.0, 1.0, num=5)
# make_reward_dense = [True, False]
terminal_state_densities = [0.25] # np.linspace(0.1, 1.0, num=5)
algorithms = ["DQN"]

print('# Algorithm, state_space_size, action_space_size, delay, sequence_length, reward_density,'
               'terminal_state_density ')
print(algorithms, state_space_sizes, action_space_sizes, delays, sequence_lengths, reward_densities,
      terminal_state_densities)


print('# Algorithm, state_space_size, action_space_size, delay, sequence_length, reward_density,'
               'terminal_state_density ')
print(algorithms, state_space_sizes, action_space_sizes, delays, sequence_lengths, reward_densities,
      terminal_state_densities)

datasets_info = np.loadtxt(stats_file + '.csv', dtype=object)
print(datasets_info[0])
print(datasets_info)
print(type(datasets_info))
print(datasets_info.shape)

import pandas as pd
stats_pd = pd.DataFrame(datasets_info)
print(stats_pd[9])

col_vals = np.array(datasets_info[:, 2:], dtype=float) # ignores 1st column - algo. name
print(col_vals)
#hack
final_rows_for_a_config = []

previous_i = 0
list_of_learning_curves = []
cols_to_take = 8
for i in range(col_vals.shape[0] - 1):

    if col_vals[i, -3] > col_vals[i + 1, -3]: #hack
#         print(col_vals[i, 6])
        list_of_learning_curves.append(col_vals[previous_i:i+1, -cols_to_take:])
        previous_i = i + 1
        final_rows_for_a_config.append(i)
print("i, previous_i:", i, previous_i)
final_rows_for_a_config.append(i + 1)
list_of_learning_curves.append(col_vals[previous_i:i + 2, -cols_to_take:])

print("final_rows_for_a_config", final_rows_for_a_config)
print("len(final_rows_for_a_config), len(list_of_learning_curves)",
        len(final_rows_for_a_config), len(list_of_learning_curves))
print(list_of_learning_curves[0])
print(list_of_learning_curves[1])
print(list_of_learning_curves[-1])
total_in_lcs = 0
for i in range(len(list_of_learning_curves)):
    print(list_of_learning_curves[i].shape)
    total_in_lcs += list_of_learning_curves[i].shape[0]
print("total_in_lcs", total_in_lcs)
# print(col_vals[final_rows_for_a_config])
final_vals = col_vals[final_rows_for_a_config]
print("final_vals.shape", final_vals.shape)
# print(final_vals[-20:, :])
metrics_ = final_vals[:, -3:]
# print(metrics_)
num_values_per_hyperparam = (1, 1, 1, 5, 4, 10, 1, 3) #hack
metrics_reshaped = np.reshape(metrics_, num_values_per_hyperparam)
print("metrics_reshaped.shape", metrics_reshaped.shape)
# print("metrics_reshaped[:, 0, 0, :, :, :, :, :]", metrics_reshaped[:, 0, 0, :, :, :, :, :].shape, metrics_reshaped[:, 0, 0, :, :, :, :, :])
to_plot_ = np.squeeze(metrics_reshaped[:, :, :, 0, 0, :, :, 1])
print(to_plot_)

# metrics_transposed = np.transpose(, ())

# print(col_vals[:,0:1])
# print(np.argwhere(col_vals[:,0:1] == [2, 2]))



In [ ]:
# loading eval metrics from csv file
# dir_name = '/home/rajanr/Backup/rl_toy'
# dir_name = '/home/rajanr/custom-gym-env' #run
# stats_file = dir_name + '/' + 'lap_22.csv' #Name of file to which benchmark stats were written
stats_file_eval = stats_file + '_eval.csv' #Name of file to which benchmark stats were written
eval_info = np.loadtxt(stats_file_eval, dtype=float)
print(eval_info, eval_info.shape)

i = 0
hack_indices = []
for line in open(stats_file_eval):
    
    line=line.strip()
#    print(line)
    if line.startswith("#HACK"):
#         print(line, i)
        hack_indices.append(i - len(hack_indices)) # appends index of last eval in this training_iteration
    i += 1
    
print(len(hack_indices), hack_indices)
hack_indices_10 = np.array(hack_indices) - 10
print(hack_indices_10.shape, hack_indices_10)
print(np.array(hack_indices[1:]) - np.array(hack_indices[:-1]))
print("Min:", min(np.array(hack_indices[1:]) - np.array(hack_indices[:-1]))) # Some problem with Ray? Sometimes no. of eval episodes is less than 10.
final_10_evals = []
for i in range(len(hack_indices)):
    final_10_evals.append(eval_info[hack_indices_10[i]:hack_indices[i]])
#     print(final_10_evals[-1])

final_10_evals = np.array(final_10_evals)
print(final_10_evals.shape, final_10_evals)

In [ ]:
# individual plots of each seeded run of train metrics at end of training
# fig = plt.figure(figsize=(9*4, 7*5)) #

# import matplotlib.pyplot as plt
# plt.imshow(np.arange(16).reshape((4, 4)), cmap='Purples', interpolation='none', extent=[2,32,2,8])
# plt.colorbar()
# # plt.xscale('log')
# # plt.yscale('log')
# # ticks = [1, 2]
# # plt.xticks(ticks)
# plt.title("ABC")
# plt.show()

print('# Algorithm, state_space_size, action_space_size, delay, sequence_length, reward_density,'
               'terminal_state_density ')
print(algorithms, state_space_sizes, action_space_sizes, delays, sequence_lengths, reward_densities,
      terminal_state_densities)

AVG = np.atleast_2d(np.squeeze(metrics_reshaped[:, 0, 0, :, :, 0, :, 1]).copy())
AVG[:] = 0

for k in range(metrics_reshaped.shape[-3]):
    for i in range(len(state_space_sizes)):
        for j in range(len(action_space_sizes)):
            to_plot_ = np.squeeze(metrics_reshaped[:, i, j, :, :, k, :, 1])
            print(np.squeeze(metrics_reshaped[:, i, j, :, :, k, :, 0]), metrics_reshaped[:, :, :, :, :, :, :, :].shape)
            AVG += to_plot_
            print(to_plot_)
            import matplotlib.pyplot as plt
            plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_))#, extent=[0, 16, 8, 14])
            plt.colorbar()
    #         plt.xscale('log')
    #         plt.yscale('log')
            plt.title(str(state_space_sizes[i]) + ' ' + str(action_space_sizes[j]))
            plt.show()

print("AVG plot")
print(AVG)
import matplotlib.pyplot as plt
plt.imshow(AVG/16, cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_))#, extent=[0, 16, 8, 14])
plt.colorbar()
#         plt.xscale('log')
#         plt.yscale('log')
plt.title('AVG plot')
plt.show()



AVG = np.squeeze(metrics_reshaped[:, :, :, 0, 0, :, :, 1].copy())
AVG[:] = 0

for i in range(len(delays)):
    for j in range(len(sequence_lengths)):
        to_plot_ = np.squeeze(metrics_reshaped[:, :, :, i, j, :, :, 1])
        AVG += to_plot_
        print(to_plot_)
        import matplotlib.pyplot as plt
        plt.imshow(to_plot_, cmap='Purples', interpolation='none', vmin=0, vmax=200)#, extent=[0, 16, 8, 14])
        plt.colorbar()
#         plt.xscale('log')
#         plt.yscale('log')
        plt.title(str(delays[i]) + ' ' + str(sequence_lengths[j]))
        plt.show()



print("AVG plot")
print(AVG)
import matplotlib.pyplot as plt
plt.imshow(AVG/16, cmap='Purples', interpolation='none', vmin=0, vmax=200)#, extent=[0, 16, 8, 14])
plt.colorbar()
#         plt.xscale('log')
#         plt.yscale('log')
plt.title('AVG plot')
plt.show()



In [ ]:
fig = plt.figure(figsize=(9*4, 7*3)) #
#ax_list = fig.axes
for i in range(len(list_of_learning_curves)):
#    for j in range(len(action_space_sizes)):
    to_plot_ = list_of_learning_curves[i][:, 0:7]
#         print(to_plot_)
    import matplotlib.pyplot as plt
    plt.plot(to_plot_[:, -2], to_plot_[:, -1], label="noise " + str(to_plot_[0, -4]) + " seed " + str(to_plot_[0, -3]))#, extent=[0, 16, 8, 14])
#        plt.colorbar()
#         plt.xscale('log')
#         plt.yscale('log')
plt.title('title')
plt.legend(loc='upper left', prop={'size': 26})
plt.show()


In [ ]:
# AVG_ = np.atleast_2d(np.squeeze(metrics_reshaped[:, 0, 0, :, :, 0, :, 1]).copy())
# AVG_[:] = 0
# temp_stack_ = []
temp_stack_.append(metrics_reshaped[:, 0, 0, :, :, :, :, 1])
print(temp_stack_[-1].shape, len(temp_stack_))

In [ ]:
#hack these 2 cells.
np_stack_ = np.concatenate(temp_stack_, axis=-1)
print(np_stack_.shape)

In [ ]:
# Plots across 10 runs: Training: with std dev across the runs
y_axis_label = 'Delay'
x_axis_label = 'Sequence length'
y_tick_labels_ = [0, 0, 1, 2, 4, 8]
x_tick_labels_ = [0, 1, 2, 3, 4]
y_axis_label = 'Transition noise'
x_axis_label = 'Reward noise'
y_tick_labels_ = [0, 0, 0.01, 0.02, 0.10, 0.25, 0.5]
x_tick_labels_ = [0, 0, 1, 5, 10, 25, 100]
# y_axis_label = 'Delay'
# x_axis_label = 'Sequence Length'
# y_tick_labels_ = ["", "0"]
# x_tick_labels_ = ["", 2, "", 3, " ", 4]

y_axis_label = ''
x_axis_label = 'Reward Density'
y_tick_labels_ = ["", ""]
x_tick_labels_ = ["", 0.25, "", 0.5, " ", 0.75]

# y_axis_label = 'n_step'
# x_axis_label = 'n_atoms'
# y_tick_labels_ = ["", 1,"", 2,"", 4,"", 8]
# x_tick_labels_ = [0, 5, 10, 20]

# y_axis_label = 'prioritized replay alpha'
# x_axis_label = 'prioritized replay beta'
# y_tick_labels_ = ["", 0.25, "", 0.5, "", 0.75, "", 1.0]
# x_tick_labels_ = [0, 0.4, 0.7, 1.0]

# y_axis_label = 'NN num layers'
# x_axis_label = 'NN layer widths'
# y_tick_labels_ = ["", 1, "", 2, "", 3, "", 4]
# x_tick_labels_ = [0, 8, 32, 128]
# x_tick_labels_ = [0, 128, 256, 512]

# y_axis_label = ''
# x_axis_label = 'NN neuron activation'
# y_tick_labels_ = ["", "", "", 2, "", 3, "", 4]
# x_tick_labels_ = ["", "tanh", "", "relu", "", "sigmoid"]

# y_axis_label = ''
# x_axis_label = 'Learning rate'
# y_tick_labels_ = ["", "", "", 2, "", 3, "", 4]
# x_tick_labels_ = ["", "1e-2", "1e-3", "1e-4", "1e-5", "1e-6", "sigmoid"]

# y_axis_label = 'DQN learning starts'
# x_axis_label = 'DQN target net update freq.'
# y_tick_labels_ = ["", 500, "", 1000, "", 2000, "", 4000]
# y_tick_labels_ = ["", 1000, "", 2000, "", 4000]
# # y_tick_labels_ = ["", 500, "1000", 2000, "4000", 8000, "", 4000]
# x_tick_labels_ = [0, 8, 80, 800]
# x_tick_labels_ = [0, 80, "", 800, "", 8000]

# y_axis_label = ''
# x_axis_label = 'Double DQN'
# y_tick_labels_ = ["", "", "", "",]
# x_tick_labels_ = [0, "False", "True"]


# y_axis_label = 'Learning rate'
# x_axis_label = 'Adam epsilon'
# y_tick_labels_ = ["", "1e-2", "1e-3", "1e-4", "1e-5", "1e-6"]
# x_tick_labels_ = ["", "1e-1", "1e-4", "1e-7", "1e-10", ""]
# x_tick_labels_ = ["", "1e-3", "1e-4", "1e-5", "1e-6", ""]


plt.rcParams.update({'font.size': 12})

for i in range(len(state_space_sizes)):
    for j in range(len(action_space_sizes)):
        mean_data_ = np.mean(metrics_reshaped[:, i, j, :, :, :, :, 1], axis=-2)
        to_plot_ = np.squeeze(mean_data_)
#         mean_data_ = np.mean(np_stack_, axis=-1)
#         to_plot_ = np.squeeze(mean_data_)
        print(np.squeeze(metrics_reshaped[:, i, j, :, :, :, :, 0]),
              metrics_reshaped[:, :, :, :, :, :, :, :].shape)
        print(to_plot_, to_plot_.shape)
        import matplotlib.pyplot as plt
        plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_) # 200 for DQN, 500 for A3C
                  )#, extent=[1, 4, 3, 0])
#         plt.xticks([1, 2])
        plt.gca().set_xticklabels(x_tick_labels_)
        plt.gca().set_yticklabels(y_tick_labels_)
        cbar = plt.colorbar()
        cbar.ax.get_yaxis().labelpad = 15
        cbar.set_label('Reward', rotation=270)
        #plt.co
#         plt.xscale('log')
#         plt.yscale('log')
#        plt.title('Final reward across Delay and Sequence length meta-features')
        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.show()
        std_dev_ = np.std(metrics_reshaped[:, i, j, :, :, :, :, 1], axis=-2)
        to_plot_ = np.squeeze(std_dev_)
#         std_dev_ = np.std(np_stack_, axis=-1)
#         to_plot_ = np.squeeze(std_dev_)
        print(to_plot_, to_plot_.shape)
        plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_)) # 60 for DQN, 100 for A3C
        #, extent=[0, 16, 8, 14])
        plt.gca().set_xticklabels(x_tick_labels_)
        plt.gca().set_yticklabels(y_tick_labels_)
        cbar = plt.colorbar()
        cbar.ax.get_yaxis().labelpad = 15
        cbar.set_label('Reward Std Dev.', rotation=270)
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.title(str(state_space_sizes[i]) + ' STD DEV ' + str(action_space_sizes[j]))
        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.show()






In [ ]:
# Same as above but for eval metrics; Plots across 10 runs: with std dev across the runs
y_axis_label = 'Delay'
x_axis_label = 'Sequence length'
y_tick_labels_ = [0, 0, 1, 2, 4, 8]
x_tick_labels_ = [0, 1, 2, 3, 4]

y_axis_label = 'Transition noise'
x_axis_label = 'Reward noise'
y_tick_labels_ = [0, 0, 0.01, 0.02, 0.10, 0.25, 0.5]
x_tick_labels_ = [0, 0, 1, 5, 10, 25, 100]

# y_axis_label = 'Delay'
# x_axis_label = 'Sequence Length'
# y_tick_labels_ = ["", "0"]
# x_tick_labels_ = ["", 2, "", 3, " ", 4]

y_axis_label = ''
x_axis_label = 'Reward Density'
y_tick_labels_ = ["", ""]
x_tick_labels_ = ["", 0.25, "", 0.5, " ", 0.75]

# final_vals = fin[final_rows_for_a_config]
# print("final_vals.shape", final_vals.shape)
# # print(final_vals[-20:, :])

print('final_rows_for_a_config', final_rows_for_a_config)
print("len(final_10_evals)", final_10_evals.shape, type(final_10_evals))
mean_data_eval = np.mean(final_10_evals, axis=1) # this is mean over last 10 eval episodes
print(mean_data_eval.shape, len(final_rows_for_a_config))
final_eval_metrics_ = mean_data_eval[final_rows_for_a_config, -2:]
print(final_eval_metrics_.shape)
num_values_per_hyperparam = list(num_values_per_hyperparam)
num_values_per_hyperparam[-1] = 2
final_eval_metrics_reshaped = np.reshape(final_eval_metrics_, num_values_per_hyperparam)
print("final_eval_metrics_reshaped.shape", final_eval_metrics_reshaped.shape)
plt.rcParams.update({'font.size': 12})

for i in range(len(state_space_sizes)):
    for j in range(len(action_space_sizes)):
        mean_data_ = np.mean(final_eval_metrics_reshaped[:, i, j, :, :, :, :, 0], axis=-2)
        to_plot_ = np.squeeze(mean_data_)
        print(np.squeeze(final_eval_metrics_reshaped[:, i, j, :, :, :, :, 0]),
              final_eval_metrics_reshaped[:, :, :, :, :, :, :, :].shape)
        print(to_plot_, to_plot_.shape)
        import matplotlib.pyplot as plt
        plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_) # 200 for DQN, 500 for A3C
                  )#, extent=[1, 4, 3, 0])
#         plt.xticks([1, 2])
        plt.gca().set_xticklabels(x_tick_labels_)
        plt.gca().set_yticklabels(y_tick_labels_)
        cbar = plt.colorbar()
        cbar.ax.get_yaxis().labelpad = 15
        cbar.set_label('Reward', rotation=270)
        #plt.co
#         plt.xscale('log')
#         plt.yscale('log')
#        plt.title('Final reward across Delay and Sequence length meta-features')
        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.show()
        std_dev_ = np.std(final_eval_metrics_reshaped[:, i, j, :, :, :, :, 0], axis=-2)
        to_plot_ = np.squeeze(std_dev_)
        print(to_plot_, to_plot_.shape)
        plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=np.max(to_plot_)) # 60 for DQN, 100 for A3C
        #, extent=[0, 16, 8, 14])
        plt.gca().set_xticklabels(x_tick_labels_)
        plt.gca().set_yticklabels(y_tick_labels_)
        cbar = plt.colorbar()
        cbar.ax.get_yaxis().labelpad = 15
        cbar.set_label('Reward Std Dev.', rotation=270)
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.title(str(state_space_sizes[i]) + ' STD DEV ' + str(action_space_sizes[j]))
        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.show()






In [ ]:
# Plot for train metrics: learning curves
#fig = plt.figure(figsize=(9*4, 7*3)) #
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
print("color_cycle", color_cycle)
seq_lens = [2, 3, 4]
plt.rcParams.update({'font.size': 15})
# metrics_reshaped_squeezed = np.squeeze(metrics_reshaped)
# print(np.squeeze(metrics_reshaped).shape)
delays = [0] + [2**i for i in range(4)]
sequence_lengths = [1, 2, 3, 4]#i for i in range(1,4)]
# transition_noises = [0, 0.01, 0.02, 0.10, 0.25]
# reward_noises = [0, 1, 5, 10, 25] # Std dev. of normal dist. #, lambda a: a.normal(0, 0.1), lambda a: a.normal(0, 0.25), lambda a: a.normal(0, 0.5),]
for i in range(len(final_rows_for_a_config)):
    if i == 0:
        to_plot_ = col_vals[0:final_rows_for_a_config[i]+1,-2]
        to_plot_x = col_vals[0:final_rows_for_a_config[i]+1,-3]
    else:
        to_plot_ = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-2]
        to_plot_x = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-3]
    if i % 10 == 0:
        fig = plt.figure(figsize=(12, 7))
    plt.plot(to_plot_x, to_plot_)#, label="Seq len" + str(seq_lens[i//10]))
    if i % 10 == 9:
#         pass
        print("Plot no.", i//10)
        plt.xlabel("Train Timesteps")
        plt.ylabel("Reward")
        plt.title('Delay ' + str(delays[i//40]) + ', Sequence Length ' + str(sequence_lengths[(i//10) % 4]))
#         plt.title('Transition Noise ' + str(transition_noises[i//50]) + ', Reward Noise ' + str(reward_noises[(i//10) % 5]))
#         plt.legend(loc='upper left', prop={'size': 26})
        plt.show()


In [ ]:
# Plot for train metrics: learning curves; with subplot
nrows_ = num_values_per_hyperparam[-5]
ncols_ = num_values_per_hyperparam[-4]
fig, ax = plt.subplots(nrows=nrows_, ncols=ncols_, figsize=(36, 21))
# print(ax)
#fig = plt.figure(figsize=(9*4, 7*3)) #
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
print("color_cycle", color_cycle)
seq_lens = [2, 3, 4]
plt.rcParams.update({'font.size': 25})
# metrics_reshaped_squeezed = np.squeeze(metrics_reshaped)
# print(np.squeeze(metrics_reshaped).shape)
delays = [0] + [2**i for i in range(4)]
sequence_lengths = [1, 2, 3, 4]#i for i in range(1,4)]
transition_noises = [0, 0.01, 0.02, 0.10, 0.25]
reward_noises = [0, 1, 5, 10, 25] # Std dev. of normal dist. #, lambda a: a.normal(0, 0.1), lambda a: a.normal(0, 0.25), lambda a: a.normal(0, 0.5),]
# sequence_lengths = [2, 3, 4]#i for i in range(1,4)]
reward_densities = [0.25, 0.50, 0.75]
for i in range(len(final_rows_for_a_config)):
    i_index = i//(10 * ncols_) # = num_seeds * shape of more frequently changing hyperparam
    j_index = (i//10) % ncols_ #
    if i == 0:
        to_plot_ = col_vals[0:final_rows_for_a_config[i]+1,-2]
        to_plot_x = col_vals[0:final_rows_for_a_config[i]+1,-3]
    else:
        to_plot_ = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-2]
        to_plot_x = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-3]
#     if i % 10 == 0:
#         fig = plt.figure(figsize=(12, 7))
#     print(i//50, (i//10) % 5)
    ax[i_index][j_index].plot(to_plot_x, to_plot_)#, label="Seq len" + str(seq_lens[i//10]))
    if i % 10 == 9: # 10 is num. of seeds
#         pass
#         print("Plot no.", i//10)
        ax[i_index][j_index].set_xlabel("Train Timesteps")
        ax[i_index][j_index].set_ylabel("Reward")
        ax[i_index][j_index].set_title('Delay ' + str(delays[i_index]) + ', Sequence Length ' + str(sequence_lengths[j_index]))
#         ax[i_index][j_index].set_title('P Noise ' + str(transition_noises[i_index]) + ', R Noise ' + str(reward_noises[j_index]))
#         ax[i_index][j_index].set_title('Sequence Length ' + str(sequence_lengths[j_index]))
#         ax[i_index][j_index].set_title('Reward Density ' + str(reward_densities[j_index]))

#         plt.legend(loc='upper left', prop={'size': 26})
fig.tight_layout()
# plt.suptitle("Training Learning Curves")
plt.show()


In [ ]:
# Plot for eval metrics: learning curves
#fig = plt.figure(figsize=(9*4, 7*3)) #
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
print("color_cycle", color_cycle)
seq_lens = [2, 3, 4]
plt.rcParams.update({'font.size': 15})
print(mean_data_eval.shape, col_vals.shape)
# metrics_reshaped_squeezed = np.squeeze(metrics_reshaped)
# print(np.squeeze(metrics_reshaped).shape)
delays = [0] + [2**i for i in range(4)]
sequence_lengths = [1, 2, 3, 4]#i for i in range(1,4)]
transition_noises = [0, 0.01, 0.02, 0.10, 0.25]
reward_noises = [0, 1, 5, 10, 25] # Std dev. of normal dist. #, lambda a: a.normal(0, 0.1), lambda a: a.normal(0, 0.25), lambda a: a.normal(0, 0.5),]
for i in range(len(final_rows_for_a_config)):
    if i == 0:
        to_plot_ = mean_data_eval[0:final_rows_for_a_config[i]+1,:]
        to_plot_x = col_vals[0:final_rows_for_a_config[i]+1,-3]
    else:
        to_plot_ = mean_data_eval[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,:]
        to_plot_x = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-3]
    if i % 10 == 0:
        fig = plt.figure(figsize=(12, 7))
    plt.plot(to_plot_x, to_plot_[:, 0])#, label="Seq len" + str(seq_lens[i//10]))
    if i % 10 == 9:
#         pass
        print("Plot no.", i//10)
        plt.xlabel("Train Timesteps")
        plt.ylabel("Reward")
        plt.title('Transition Noise ' + str(transition_noises[i//50]) + ', Reward Noise ' + str(reward_noises[(i//10) % 5]))
#         plt.legend(loc='upper left', prop={'size': 26})
        plt.show()


In [ ]:
# Plot for eval metrics: learning curves; with subplots
fig, ax = plt.subplots(nrows=nrows_, ncols=ncols_, figsize=(36, 21))
#fig = plt.figure(figsize=(9*4, 7*3)) #
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
print("color_cycle", color_cycle)
seq_lens = [2, 3, 4]
plt.rcParams.update({'font.size': 25})
print(mean_data_eval.shape, col_vals.shape)
# metrics_reshaped_squeezed = np.squeeze(metrics_reshaped)
# print(np.squeeze(metrics_reshaped).shape)
delays = [0] + [2**i for i in range(4)]
sequence_lengths = [1, 2, 3, 4]#i for i in range(1,4)]
transition_noises = [0, 0.01, 0.02, 0.10, 0.25]
reward_noises = [0, 1, 5, 10, 25] # Std dev. of normal dist. #, lambda a: a.normal(0, 0.1), lambda a: a.normal(0, 0.25), lambda a: a.normal(0, 0.5),]
# sequence_lengths = [2, 3, 4]#i for i in range(1,4)]
reward_densities = [0.25, 0.50, 0.75]
for i in range(len(final_rows_for_a_config)):
    i_index = i//(10 * ncols_) # = num_seeds * shape of more frequently changing hyperparam
    j_index = (i//10) % ncols_ #
    if i == 0:
        to_plot_ = mean_data_eval[0:final_rows_for_a_config[i]+1,:]
        to_plot_x = col_vals[0:final_rows_for_a_config[i]+1,-3]
    else:
        to_plot_ = mean_data_eval[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,:]
        to_plot_x = col_vals[final_rows_for_a_config[i-1]+1:final_rows_for_a_config[i]+1,-3]
#     if i % 10 == 0:
#         fig = plt.figure(figsize=(12, 7))
#     plt.plot(to_plot_x, to_plot_[:, 0])#, label="Seq len" + str(seq_lens[i//10]))
    ax[i_index][j_index].plot(to_plot_x, to_plot_[:, 0])#, label="Seq len" + str(seq_lens[i//10]))
    if i % 10 == 9: # 10 is num. of seeds
#         pass
#         print("Plot no.", i//10)
        ax[i_index][j_index].set_xlabel("Train Timesteps")
        ax[i_index][j_index].set_ylabel("Reward")
        ax[i_index][j_index].set_title('Delay ' + str(delays[i_index]) + ', Sequence Length ' + str(sequence_lengths[j_index]))
#         ax[i_index][j_index].set_title('P Noise ' + str(transition_noises[i_index]) + ', R Noise ' + str(reward_noises[j_index]))
#         ax[i_index][j_index].set_title('Sequence Length ' + str(sequence_lengths[j_index]))
#         ax[i_index][j_index].set_title('Reward Density ' + str(reward_densities[j_index]))
#         plt.legend(loc='upper left', prop={'size': 26})
fig.tight_layout()#pad=1, w_pad=1, h_pad=2.0)
plt.show()


In [ ]:
for i in range(len(state_space_sizes)):
    for j in range(len(action_space_sizes)):
        for k in range(10):
            mean_data_ = np.mean(metrics_reshaped[:, i, j, :, :, k, :, 1], axis=0)
            to_plot_ = np.squeeze(mean_data_)
            print(to_plot_, to_plot_.shape)
            plt.imshow(np.atleast_2d(to_plot_), cmap='Purples', interpolation='none', vmin=0, vmax=200)#, extent=[0, 16, 8, 14])
            plt.colorbar()
    #         plt.xscale('log')
    #         plt.yscale('log')
            plt.title(str(state_space_sizes[i]) + ' ' + str(action_space_sizes[j]))
            plt.show()


In [ ]:
#####INCOMPLETE
# 2-D heatmaps
#
# 
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

# alg_name = 'DQN'

# measure = 'time'
# measure = 'memory'
# dataset_index = np.where(dataset_names==dataset_name)[0][0]
# print(type(dataset_index), dataset_index.dtype)
# column_no = 19 # Refer to comments above to decide the benchmark column or measure to choose

fig = plt.figure(figsize=(9*4, 7*5)) #


# column_names = ['train', '\"val\"','test_1','test_4']
# column_nos = np.arange(4, 22, 5) #set starting value to 4 for performances, 5 for scoring times
# print(column_nos.shape, dataset_names.shape)
for i in range(len(state_space_sizes)):
    for j in range(len(action_space_sizes)):
        for k in range(len(delays)):
            for l in range(len(sequence_lengths)):
                print("Plotting for |S|, |A|, d, n: ", state_space_sizes[i], action_space_sizes[j], delays[k]
                     sequence_lengths[l])
                values_[0, i, j]

        

plt.savefig('/home/rajanr/Downloads/temp.png')


In [ ]:
num_ = 5
len_ = 3
nums_ = list(range(num_ + 1 - len_, num_ + 1))
print(nums_, np.random.randint(np.prod(nums_)))
unique_seqs_ = []
for i in range(np.prod(nums_)):
    curr_num = i
    seq_ = []
    curr_rem_nums_ = list(range(num_))
    for j in nums_[::-1]:
        rem_ = curr_num % j
        seq_.append(curr_rem_nums_[rem_])
        del curr_rem_nums_[rem_]
#         print("curr_rem_nums_", curr_rem_nums_)
        curr_num = curr_num // j
#         print(rem_, curr_num, j, seq_)
    print("T/F:", seq_ in unique_seqs_)
    unique_seqs_.append(seq_)
#print(len(set(unique_seqs_))) #error

print(np.unique(unique_seqs_))
#         print(i % 3, i % 4, i % 5, i)




In [ ]:
np.random.seed(0)
np.random.choice(5)

In [ ]:
#temp

new_file = "/home/rajanr/temp.txt"
fout = open(new_file, 'a') #hardcoded
for line in open(stats_file):
    if line.startswith("#"):
        
        vals = line.split(',')[-1]
#         print(vals)
    else:
        vals2 = line
#         print(line)
    
        print(vals[1:-1] + ' ' + vals2)
        fout.write(vals[1:-1] + ' ' + vals2)
# # fout.flush()
fout.close()


In [ ]:
from os import getcwd as pwd
from glob import glob #ls = sorted(glob(filename))
from ray.tune.analysis import ExperimentAnalysis

for filename in sorted(glob('/mhome/rajanr/ray_results/DQN/*.json')):
    print(filename)
    try:
        print(ExperimentAnalysis(filename, trials=None))
    except Exception as e:
        print("EXCEPTION", filename, e)